In [46]:
from model import *
from torch_utils import *
import requests
import json

# Create and train the model

In [47]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

# print(text[:1000])

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

data = torch.tensor(encode(text), dtype=torch.long)

max_iters = 500
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
# data loading



model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


print("DATE : ")
print(datetime.now())

train_model(m, train_data, val_data, block_size, batch_size, 5000, learning_rate, eval_iters, eval_interval)
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))

print("DATE : ")
print(datetime.now())

length of dataset in characters:  1115394

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
0.209729 M parameters
DATE : 
2024-07-31 14:34:20.900029
step 0: train loss 4.3289, val loss 4.3329
step 100: train loss 2.6524, val loss 2.6671
step 200: train loss 2.5064, val loss 2.5083
step 300: train loss 2.4296, val loss 2.4404
step 400: train loss 2.3474, val loss 2.3469
step 500: train loss 2.3066, val loss 2.3163
step 600: train loss 2.2570, val loss 2.2657
step 700: train loss 2.2215, val loss 2.2318
step 800: train loss 2.1742, val loss 2.1984
step 900: train loss 2.1274, val loss 2.1637
step 1000: train loss 2.1290, val loss 2.1511
step 1100: train loss 2.0831, val loss 2.1233
step 1200: train loss 2.0576, val loss 2.1025
step 1300: train loss 2.0258, val loss 2.0696
step 1400: train loss 1.9978, val loss 2.0578
step 1500: train loss 1.9756, val loss 2.0498
step 1600: train loss 1.9677, val loss 2.0312
step 1700: train loss 1.9381, val loss 2.0206
step 1800: train

# Save the weights

In [48]:
block_weights = [block.state_dict() for block in m.blocks]
torch.save(block_weights, 'block_weights.pth')

# Distributed Block

In [49]:
def call_block_forward(block, input_tensor):
    url = f"http://127.0.0.1:8000/block{block}/forward"
    headers = {"Content-Type": "application/json"}
    data = {"input_tensor": input_tensor.tolist()}
    # print(input_tensor)
    # print("PRE POST")
    response = requests.post(url, headers=headers, data=json.dumps(data))
    # print("POST POST")
    
    if response.status_code == 200:
        output = response.json()["output"]
        return output
    else:
        print(f"Error: {response.status_code}")
        return None
class DistribBlock(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, block_num):
        super().__init__()
        self.block_num = block_num
        
    def forward(self, x):
        res = call_block_forward(self.block_num, x)
        tensor = torch.tensor(res, dtype=torch.float32).to(device)
        return tensor

In [50]:
b1 = DistribBlock("1")
b2 = DistribBlock("2")
b3 = DistribBlock("3")
b4 = DistribBlock("4")

In [51]:
m.blocks = nn.Sequential(b1, b2, b3, b4)

In [52]:
print(sum(p.numel() for p in m.parameters()), 'parameters') # 10561 vs 209729 for the full one

10561 parameters


In [55]:
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))


HORTo beausd.

POMPEY:
Lethere belione thou day: where to me's I thung conspiught of this contrangeds on.
What joy, who and: what hisfency to doth then ofther bothoutians did,
Syeed from this to him s
